# IBM Business Automation Workflow recommendation service with IBM Business Automation Insights and Watson Machine Learning

Artificial intelligence can be combined with business processes management in many ways. For example, AI can help transforming unstructured data into data that a process can work with, through techniques such as visual or text recognition. Assistants and bots provide a better user experience and several IBM Watson services can help achieve those goals but a business process can capture lots of business data. This notebook demonstrates how to take better benefit of this data and inject machine-learning techniques to optimize processes. If for every decision that needs to be taken as part of a business process you can get a recommendation based on the decisions that have been taken in the past in similar situations, your processes are greatly enhanced. 

## The recommendation service scenario

The scenario here is the following: imagine an insurance company which has set up a Workflow process to approve or reject insurance claims. Some of those insurance claims are simple because, typically, the amount is small and the customer's history and claim circumstances are straightforward. Such claims can be approved automatically or, at least, follow a fast approval path. Some claims are more complex and therefore their approval path includes more steps. Let us further assume that the approval decision or the decision on which path to follow is a human task and this task is captured as a task of a Workflow process. Then it becomes interesting to consider whether a machine-learning algorithm can help figure out which decision to take, based on past decisions.
This scenario can be adapted to any kind of human decision process. In the insurance claim example, the decision consists in approving or rejecting a claim, which amounts to a kind of yes-or-no decision. Such decisions can translate into a binary classification machine-learning problem. However, if the decision consists in dispatching a process into many other subprocesses, the scenario becomes a multiclass classification problem, which Business Automation Insights can also handle.

This recommendation service uses IBM Business Automation Workflow to build the claim approval process, IBM Business Automation Insights to store the business process data, and IBM Watson for artificial intelligence, in particular Watson Studio for building the machine-learning model and Watson Machine Learning for deploying that model. 


## Overview of the solution

First, a schema outlines how all the different elements and cloud services are used together to build the expected service.

![](https://raw.githubusercontent.com/Tissandier/bpm-recommendation/master/images/overview.png)


Everything starts with the business process itself, which runs in <b>IBM Business Automation Workflow</b>. As the process is running, the business data of the process, which in our scenario contains information about the insured person and the claim, is captured by the <b>Business Automation Insight (BAI)</b> service, which stores all the process operational data and in particular our claim data in HDFS. The role of this BAI service is really to capture and store this data so that the processes can be monitored and, as the name indicates, provides you with insights on the process. BAI can render various dashboards, for example to monitor the process efficiency. In this insurance claim scenario, you are more interested in the data that is associated with activities and processes rather than in the operational data. <br>
Once the data is captured in <b>HDFS</b>, it can be used to train a machine-learning model. After the model is trained with existing claim data and approval decisions, it should be able to provide recommendations on whether to approve or reject new claims.<br>
The trained model needs to be deployed, which is the role of the <b>IBM Watson Machine Learning</b> service. This service stores the machine-learning model and provides a scoring endpoint.
Finally, the scoring endpoint can be invoked by the Workflow business-management process and the result transformed into a recommendation within the process user interface.


## Learning goals

In this notebook, you will learn:

- How to load time series data, in IBM Business Automation Insights, from a specific tracking point in the Workflow process
- How to explore the format of the data and read it
- How to create an Apache® Spark machine learning pipeline, which will be the recommendation model
- How to train and evaluate the model.
- Persist a pipeline and model in Watson Machine Learning repository.
- Deploy a model for online scoring using Wastson Machine Learning API.
- Score sample scoring data by using the Watson Machine Learning API.
- How to set up the scoring to create a recommendation service in a Workflow Coach



## Setting up the solution

To illustrate how to combine all the technologies together, the notebook comes with a business process definition that you can download from here: [Download the BPM Process](https://github.com/Tissandier/bpm-recommendation/raw/master/process/Claim_Approval_Sample.twx)

To be able to run the solution that is presented in this notebook, make sure the following elements are installed:

- IBM Business Automation Workflow

- IBM Business Automation Insight 

Business Automation Insight must be installed and connected to an HDFS data lake.

- IBM Watson Machine Learning service (https://console.bluemix.net/catalog/services/machine-learning) on IBM Cloud 
You can install a free tier.

Once you have the installed the various elements, ensure you have :
- Credentials for your Workflow instance
- Credentials for the HDFS used by Business Automation Insights
- Watson Machine Learning credentials

Please note the notebook requires a Python 3.5 and Spark 2.1 kernel to run.

## Tracking data in Business Automation Insights
Download and import the process definition from the Workflow Center.
<br>
![](https://raw.githubusercontent.com/Tissandier/bpm-recommendation/master/images/importprocess.png)
Then open the 'Claim Approval Sample' process application in Process Designer. As you explore the process application, you see one 'Claim approval' process, which has been defined as a single user task.
<br>
![](https://raw.githubusercontent.com/Tissandier/bpm-recommendation/master/images/process.png)
<br>
<br>
For this process, four classes of business data have been created: claim, customer, vehicle, and recommendation. The 'claim' business data represents the data of the insurance claim. It will reference a customer and a vehicle. The 'recommendation' object will contain information from the recommendation service that's being built. This object will be examined later.<br>
Note that this example is not intended to reflect a real claim approval system, which is notably more complex.
The claim contains information on the vehicle: the 'make', the 'type' and 'model', and the 'year' of the vehicle, information about the customer, in particular the 'creditScore' property, which represents the customer's insurance score, and information about the claim itself such as the estimated amount, the assessment that was made, and the assessor. The example uses only some of this information. <br>
<br>
Since this is not a real process, we initialize the claim object with some random data.
<br>
![](https://raw.githubusercontent.com/Tissandier/bpm-recommendation/master/images/claimdata.png)
<br>
The main task in this process is to approve or reject an insurance claim and thus to decide (based on the claim data) whether to set the 'approved' attribute of the claim to 'true' or to 'false'. 

After the approval decision is taken --that is, when the approval task is finished-- this piece of information is stored in Business Automation Insights so that it can be fed to the machine-learning model. For this purpose, a 'tracking point' is introduced after the approval task. 
The tracking point in a process is a moment when all the current status and data is sent to Business Automation Insights. 
Each tracking point can store the appropriate data. This example stores the data of the claim that the machine algorithm is to learn from. Of course, the decision value of the 'approved' property of the claim is stored, too.
<br>
![](https://raw.githubusercontent.com/Tissandier/bpm-recommendation/master/images/tracking.png)
<br>
Each tracking point stores the information that has been specified when a tracking group has been created. The tracking group is really a model of the data that needs to be stored in BAI.<br>
The tracking point definition specifies the tracking group and the mapping from the claim data to the tracking group data.
<br>
Also note the name of the tracking group: IBMBPMRSTraining_Claims, which is necessary to find the data in the next step.  


## Creating some data to train the system
At this point, it is necessary to create some data to train the system. You can continue the exercise even with few data, but you have to run the process from Process Portal 10 to 20 times.<br>
As you run the process, you can see the coach making some recommendations for you. Because at the moment no recommendation service has been created yet, those recommendations are fake.<br>
However, you should still follow those recommendations when you create the initial data because by doing so, you will create a set of initial data for which the machine-learning model will be easy to create. 

If you want to get directy data, the alternative is to download [the training file](https://raw.githubusercontent.com/Tissandier/bpm-recommendation/master/data/sample_training_data.json) and place this file in your HDFS system. You will then have to make sure you update the part of the code that reads the BAI data so that it points to this file instead of the BAI data path.


## The format of the Business Automation Insights data
After the process has run several times, events are stored in Business Automation Insights. BAI stores many different types of events but in this scenario, the events that are registered when a tracking point is reached by a process are stored as a 'bpm-timeseries' for tracking data.
Every time a process is going through the tracking point, a record is added to HDFS in the form of JSON data.<br>
In this scenario, the timeseries data is partitioned by the following elements:
- The identifier and version number of the Workflow business process application
- The tracking group identifier 

Thus, HDFS file names start with the following path:<br>
<br>
[hdfs root]/ibm-bai/bpmn-timeseries/[processAppId]/[processAppVersionId]/tracking/[trackingGroupId]
<br>
<br>
Remember, the tracking group name is IBMBPMRSTraining_Claims. To find the data, you query the various ids from the Workflow system


## How to find an application id and version, and the tracking group id

In this example, when the process is imported into the Workflow instance, the process application IDs and versions, and the tracking group ID, do not change, Therefore, to run the example, predefined IDs could be used but the demo shows how you can retrieve all the IDs by using the IBM Workflow REST API<br>
<br>You may skip this part and go directly to the next chapter.
<br>You can also refer to https://www.ibm.com/support/knowledgecenter/SS8JB4_18.0.0/com.ibm.wbpm.ref.doc/rest/bpmrest/rest_bpm_wle.htm to get more details on the REST API that is used below.<br>

Here is some Python code to set up the REST API URL. All you have to do is modify this code to specify the correct host name and credentials to access the process server REST API. Make sure you <strong>change the host and credentials to your Workflow credentials</strong>. Your Workflow system might not be accessible from this notebook. In this case, copy-paste this code and run it in a system that can access your Workflow environment.


In [ ]:
import urllib3, requests, json
bpmusername='admin'
bpmpassword='admin'
bpmrestapiurl = 'https://localhost:9443/rest/bpm/wle/v1'

headers = urllib3.util.make_headers(basic_auth='{username}:{password}'.format(username=bpmusername, password=bpmpassword, verify=False))


Now you retrieve the process application ID and version number by using the 'processApps' REST API. The code below searches for the 'Claim Approval Sample' application and assumes that only one version or snapshot is installed.

In [ ]:
url = bpmrestapiurl + '/processApps'
response = requests.get(url, headers=headers, verify=False)

[processApp] = [x for x in json.loads(response.text).get('data').get('processAppsList') if x.get('name') == 'Claim Approval Sample']
processAppId = processApp.get('ID')

# Note that the 5 first characters of the process app id below are removed
# because the REST API returns the process application id with a 5-letter prefix that is '2066.'.
# This prefix marks the identifier as a process application id but you won't need this prefix later.

print("the process application id: " + processAppId[5:])
snapshot = processApp.get('installedSnapshots')[0]
processAppVersionId = snapshot.get('ID')
print("the process application version id: " + processAppVersionId)

You can now retrieve the tracking group ID. For this, you call the BPM 'assets' API by using the versionId (or snapshot ID) that has just been computed. Assets are filtered so that only the definitions of the tracking group are retrieved. For this purpose, another call to the Workflow 'assets' API is necessary, using the version or snapshot identifier that has just been computed.


In [ ]:
url = bpmrestapiurl + '/assets'
response = requests.get(url, headers=headers, verify=False, params={'processAppId': processAppId, 'filter': 'type=TrackingGroup' })

[trackingGroupId] = [x.get('poId') for x in json.loads(response.text).get('data').get('TrackingGroup') if x.get('name') == 'IBMBPMRSTraining_Claims']


# Note that the 3 first characters of the tracking group id below are removed
# because the REST API returns the tracking group id with a 3-letter prefix that is '14.'.
# This prefix marks the identifier as a tracking group id but you won't need this prefix later.

print('The tracking group id : ' + trackingGroupId[3:])


Now you know the processApp id and version, and the tracking group id, so that you can query data.


## Using Spark SQL to read Business Automation Insights data
Business Automation Insights stores data in HDFS. As described above, the events coming from the Workflow instance are stored in JSON files. At this point, you must adapt the code below to specify the HDFS URL.

In [ ]:
from pyspark.sql import  SparkSession

hdfs_root = 'hdfs://your hdfs root here'

processAppId = '638d314f-12db-43c3-9051-89f3ce992393'
processAppVersionId = '2064.4310cecf-969e-48ce-9ac3-00e73de5dfb9'
trackingGroupId = 'f1cf87ab-29ae-4b54-901a-6601b4539132'

spark = SparkSession.builder.getOrCreate()
spark.conf.set("dfs.client.use.datanode.hostname", "true")

try:
  timeseries = spark.read.json(hdfs_root + "/ibm-bai/bpmn-timeseries/" + processAppId + '/' + processAppVersionId + '/tracking/' + trackingGroupId + '/*/*')
  timeseries.createOrReplaceTempView("timeseries")
  timeseries.show()
  timeseries.printSchema()
  print ('The data containts ' + str(timeseries.count()) + ' events')
except:
  print('Exception while reading data, please ensure data was created in BAI')

Note that the various ids for the path are specified in the JSON path. This HDFS path could also use HDFS wildcards. Here, the * character replaces any directory or file name in the path.

In [ ]:
businessdata = spark.sql("SELECT trackedFields.* from timeseries")
businessdata.show()
businessdata.printSchema()

## Create an Apache® Spark machine-learning model

Watson Machine learning supports a growing number of IBM or open-source machine-learning and deep-learning packages. This example uses Spark ML and in particular the Random Forest Classifier algorithm. Learn now how to prepare data, create an Apache® Spark machine-learning pipeline, and train the model.

In [ ]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline, Model

### Adaptation of data

The following code rename the columns to remove the type from it.<br>
Then, the StringIndexer method transforms the 'approved' column, which is a column of type 'string' containing only 'true' or 'false' values, into a numeric column with '0' and '1' values so that the classifier can understand it.<br>

The VectorAssembler class creates a new features column which contains the features from which to build the model.<br>
The IndexToString method transforms the prediction/classification of the model, which will be '0' and '1' values, back into "true" or "false" strings.

In [ ]:
businessdata = businessdata.withColumnRenamed("approved.string", "approved")
businessdata = businessdata.withColumnRenamed("creditScore.integer", "creditScore")
businessdata = businessdata.withColumnRenamed("estimateAmount.integer", "estimateAmount")
businessdata = businessdata.withColumnRenamed("approvedAmount.integer", "approvedAmount")

features = ["approvedAmount", "creditScore", "estimateAmount"]
approvalColumn = "approved"


approvalIndexer = StringIndexer(inputCol='approved', outputCol="label").fit(businessdata)

assembler = VectorAssembler(inputCols=features, outputCol="features")

labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=approvalIndexer.labels)

###  Creating the model
The model is built from the RandomForestClassifier algorithm.

In [ ]:
rf = RandomForestClassifier(labelCol="label", featuresCol="features")

In the cell below the data is splitted into training data and test data and the prediction model is trained and then tested, finally the accuracy of the model is displayed.

In [ ]:
businessdata = businessdata[features+['approved']]
splitted_data = businessdata.randomSplit([0.8, 0.20], 24)
train_data = splitted_data[0]
test_data = splitted_data[1]

pipeline = Pipeline(stages=[approvalIndexer, assembler, rf, labelConverter])

model = pipeline.fit(train_data)

predictions = model.transform(test_data)
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

print("Accuracy = %g" % accuracy)
print("Test Error = %g" % (1.0 - accuracy))


## Storing the model in Watson ML
Watson machine learning is used here to store the resulting model. After the model is stored , Watson machine learning makes it possible to create an HTTP scoring endpoint, which is then used as the recommendation service.
The code below stores the created model and pipeline in Watson Machine Learning. Note that you need to <b>specify the authentication information from your instance of Watson Machine Learning service</b> in the code below.


In [ ]:
!rm -rf $PIP_BUILD/watson-machine-learning-client
!pip install --upgrade watson-machine-learning-client==1.0.260

In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

# Authenticate to Watson Machine Learning service on Bluemix.

wml_credentials={
  "url": "https://ibm-watson-ml.mybluemix.net",
  "access_key": "place acces key here",
  "username": "place username here",
  "password": "place password here",
  "instance_id": "place instance id key here"
}
# wml_service_path, user and wml_password can be found on Service Credentials tab of service instance created in Bluemix.

client = WatsonMachineLearningAPIClient(wml_credentials)

We can now save the model and the training data. 

In [ ]:
published_model_details = client.repository.store_model(model=model, meta_props={'name':'Claim Approval Recommendation Model'}, training_data=train_data, pipeline=pipeline)

Here is the list of the models that are currently stored in the machine learning instance

In [ ]:
client.repository.list_models()

<a id="scoring"></a>
## Deploying the model

Now that the model is stored, we need to deploy it in a runtime environement, we start by retrieving the model uid:


In [ ]:
model_uid = client.repository.get_model_uid(published_model_details)
print(model_uid)

Below we list already installed deployments, A free tier in Watson machine learning allows no more than five deployments.

In [ ]:
print(client.deployments.get_uids())

We use the deployments client API to create a new deployment for our model:

In [ ]:
deployment_details = client.deployments.create(asset_uid=model_uid, name='Recommendation Prediction Model')

As part of the deployment details result is the URL that will allow us to score against the published model.

In [ ]:
recommendation_url = client.deployments.get_scoring_url(deployment_details)
print(recommendation_url)

### Testing the recommendation URL

You can test the scoring URL with some data to see how it works. The next step consists in using this URL from within the Workflow process.

In [ ]:
import json
recommendation_data = {"fields": ["approvedAmount", "creditScore", "estimateAmount"],"values": [[2000, 500, 2000]]}

scoring_response = client.deployments.score(recommendation_url, recommendation_data)

print(json.dumps(scoring_response, indent=3))

## Invoking the recommendation REST endpoint from the Workflow process


To display a recommendation for a decision on a claim within the Workflow process itself, invoke the Recommendation service from a Workflow service. 
<br>
If you go back to the installed process application, you can see a service flow called 'Invoke Watson ML Service Flow'. This is the service that calls the recommendation REST endpoint .
<br>
![](https://raw.githubusercontent.com/Tissandier/bpm-recommendation/master/images/invocationscript.png)
<br>
As soon as you specify your credentials to Watson Machine Learning and the recommendation URL in this script, the Workflow process displays recommendations from the Spark machine-learning model.

<br>
The result of the recommendation service is displayed in the process UI (the coach) after the service has been called. In the picture below, you see that the coach contains two different parts, one for the 'I recommend' and another one for 'I do not recommend', the visibility of each portion depends on the result of the recommendation service.

![](https://raw.githubusercontent.com/Tissandier/bpm-recommendation/master/images/coach.png)

The system is now ready to return recommendations about the insurance claim.

![](https://raw.githubusercontent.com/Tissandier/bpm-recommendation/master/images/reco.png)


### Conclusion

This notebook is designed to help you understand how to create a recommendation service for your Workflow Process with Watson Machine Learning and Business Automation Insights. You are encouraged to explore more possibilities of Watson Studio and Watson Machine Learning, in particular the capability to retrain the model when more data becomes available.


Author: Emmanuel Tissandier is a Senior Technical Staff Member and architect in the Business Automation team in the IBM France Lab.